In [ ]:
import numpy as np
import pyrealsense2 as rs
from PIL import Image, ImageDraw
from transformers import Owlv2Processor, Owlv2ForObjectDetection
import torch
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the processor and model.
processor = Owlv2Processor.from_pretrained("google/owlv2-base-patch16-ensemble")
model = Owlv2ForObjectDetection.from_pretrained("google/owlv2-base-patch16-ensemble")

In [ ]:
# RealSense 파이프라인 시작
pipeline = rs.pipeline()
config = rs.config()

# 색상과 깊이 스트림 활성화 
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30) 
config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30)

# 파이프라인 시작
profile = pipeline.start(config)

# 필터 설정
spatial = rs.spatial_filter()        # Spatial Edge-Preserving filter
#temporal = rs.temporal_filter()      # Temporal filter

# Spatial filter 설정
spatial.set_option(rs.option.holes_fill, 5)        # Hole filling
spatial.set_option(rs.option.filter_smooth_alpha, 0.5)  # Alpha (0 ~ 1)
spatial.set_option(rs.option.filter_smooth_delta, 20)   # Delta (1 ~ 50)

# depth-to-disparity 변환 필터 추가
depth_to_disparity = rs.disparity_transform(True)
disparity_to_depth = rs.disparity_transform(False)

# Temporal filter 설정
#temporal.set_option(rs.option.filter_smooth_alpha, 0.4)  # Alpha (0 ~ 1)

# 프레임 세트 수집
frameset = pipeline.wait_for_frames()

# 깊이 프레임을 기준으로 정렬
align = rs.align(rs.stream.color)
frameset = align.process(frameset)

# 정렬된 컬러 및 깊이 프레임 가져오기
aligned_depth_frame = frameset.get_depth_frame()  # 정렬된 깊이 프레임
aligned_color_frame = frameset.get_color_frame()  # 정렬된 컬러 프레임

# Depth를 Disparity로 변환
disparity_frame = depth_to_disparity.process(aligned_depth_frame)

# Spatial Filter 적용 (edge-aware)
filtered_depth = spatial.process(disparity_frame)


# Temporal filter 적용
#filtered_depth_frame = temporal.process(filtered_depth_frame)

# 다시 Depth로 변환
depth_frame_filtered = disparity_to_depth.process(filtered_depth)

# 필터가 적용된 깊이 프레임 데이터를 NumPy 배열로 변환
final_depth_frame = np.asanyarray(depth_frame_filtered.get_data())

# 정렬된 컬러 프레임을 NumPy 배열로 변환
aligned_color_image = np.asanyarray(aligned_color_frame.get_data())

# 이미지의 밝기 조절 (픽셀 값 증가)
# 이미지를 밝게 하려면, 원본 이미지에 일정 값을 더함. 이 예시에서는 50을 더함.
#brightness_factor = 50
#bright_image = cv2.convertScaleAbs(aligned_color_image, alpha=1, beta=brightness_factor)

# PIL로 변환하여 matplotlib을 사용해 시각화
blurred_image_rgb = cv2.cvtColor(aligned_color_image, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 변환
inverted_image = Image.fromarray(blurred_image_rgb)

# 이미지 표시
plt.imshow(inverted_image)
plt.show()

# 파이프라인 정지
pipeline.stop()

In [ ]:
# 이미지와 텍스트 쿼리를 모델에 입력
texts = [['box']]  # 원하는 물체에 대한 쿼리
inputs = processor(text=texts, images=inverted_image, return_tensors="pt")

# OWLv2 모델을 사용하여 객체 감지 수행
with torch.no_grad():
    outputs = model(**inputs)
    
# 모델 출력 후처리
target_sizes = torch.Tensor([inverted_image.size[::-1]])  # 이미지 크기를 사용하여 대상 크기 정의
results = processor.post_process_object_detection(outputs=outputs, target_sizes=target_sizes, threshold=0.2)

# 결과에서 boxes, scores, labels 추출
if results:
    boxes, scores, labels = results[0]["boxes"], results[0]["scores"], results[0]["labels"]
else:
    print("No objects detected")
    boxes, scores, labels = [], [], []

# 깊이와 픽셀 값 저장할 리스트 초기화
depths = []
pixels = []  # 픽셀 좌표를 저장할 리스트

# 경계 상자와 깊이 정보 추출 및 처리
for box, score, label in zip(boxes, scores, labels):
    box = [round(i, 2) for i in box.tolist()]
    x1, y1, x2, y2 = tuple(box)

    # 이미지에 경계 상자 그리기
    draw = ImageDraw.Draw(inverted_image)
    draw.rectangle(xy=((x1, y1), (x2, y2)), outline="red")
    draw.text(xy=(x1, y1), text=texts[0][label], fill='black')

    # 경계 상자의 중심 좌표 계산
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    # 중심 좌표에 해당하는 깊이 값 추출
    depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
    if 0 <= int(center_y) < final_depth_frame.shape[0] and 0 <= int(center_x) < final_depth_frame.shape[1]: #center_x, center_y가 배열 범위를 벗어나는 경우에 대비해 범위 검사 추가
        center_depth_value = final_depth_frame[int(center_y), int(center_x)].astype(float) * depth_scale

    # 깊이 값 출력
    print(f"Detected a {texts[0][label]} {center_depth_value:.3f} meters away.")
    
    # 중심 좌표와 깊이값을 픽셀 리스트에 추가
    bounding_box_pixels = [center_x, center_y, round(center_depth_value, 3)]
    pixels.append(bounding_box_pixels)

# Deprojection 함수
def rs2_deproject_pixel_to_point(intrin, pixels):
    points = []
    
    # 각 픽셀 좌표와 해당 깊이 값을 사용해 Deprojection 수행
    for i in range(len(pixels)):
        pixel = pixels[i]
        depth = pixels[i][2]  # 각 픽셀에 해당하는 깊이 값

        x = (pixel[0] - intrin['ppx']) / intrin['fx']
        y = (pixel[1] - intrin['ppy']) / intrin['fy']

        if intrin['model'] == 'RS2_DISTORTION_INVERSE_BROWN_CONRADY':
            r2 = x * x + y * y
            f = 1 + intrin['coeffs'][0] * r2 + intrin['coeffs'][1] * r2 * r2 + intrin['coeffs'][4] * r2 * r2 * r2
            ux = x * f + 2 * intrin['coeffs'][2] * x * y + intrin['coeffs'][3] * (r2 + 2 * x * x)
            uy = y * f + 2 * intrin['coeffs'][3] * x * y + intrin['coeffs'][2] * (r2 + 2 * y * y)
            x = ux
            y = uy

        # Deprojection 결과로 3D 포인트를 구함
        point = np.array([depth * x, depth * y, depth])
        points.append(point)  # 각 포인트를 리스트에 추가

    return points

# Example usage
#intrin = {
#    'model': 'RS2_DISTORTION_INVERSE_BROWN_CONRADY',
#    'ppx': 424.9087219238281,
#    'ppy': 236.51449584960938,
#    'fx': 420.6676330566406,
#    'fy': 420.6676330566406,
#    'coeffs': [0, 0, 0, 0, 0]
#}

# Example usage
intrin = {
    'model': 'RS2_DISTORTION_INVERSE_BROWN_CONRADY',
    'ppx': 964.6163330078125,
    'ppy': 574.3912963867188,
    'fx': 1366.9173583984375,
    'fy': 1364.696533203125,
    'coeffs': [0, 0, 0, 0, 0]
}

for i in range(len(pixels)):
    
    # 각 픽셀 좌표에 대해 Deprojection 수행
    points=rs2_deproject_pixel_to_point(intrin, pixels)
print(points)

plt.imshow(inverted_image)
plt.show()

In [ ]:
# Define rigid body transformation function
def apply_rigid_body_transform(point, rotation_matrix, translation_vector):
    # Convert point to numpy array if not already
    point = np.array(point)

    # Apply rigid body transformation: R * point + T
    transformed_point = np.dot(R, point) + T
    return transformed_point

# Example extrinsics from depth to color stream
rotation_matrix = [
    0.999944269657135, -0.00910048745572567, -0.005356862209737301,
    0.009115878492593765, 0.999954342842102, 0.002855878323316574,
    0.005330628249794245, -0.0029045515693724155, 0.9999815821647644
]

# Reshape rotation matrix to 3x3 matrix
R = np.array(rotation_matrix).reshape(3, 3) #rotation_matrix를 미리 numpy array로 변환해, 성능에 유리하도록 함.

translation_vector = [-0.014955335296690464, -3.531932452460751e-05, -0.00015003549924585968]

# Convert translation vector to numpy array
T = np.array(translation_vector) #미리 할당된 크기의 numpy 배열로 저장하면 반복문 안에서 메모리 할당이 줄어들어 성능에 유리하게 만들 수 있음.

# 리스트 초기화
transformed_points = []

# 각 3D 포인트에 대해 강체 변환을 적용
for point in points:
    transformed_point = apply_rigid_body_transform(point, rotation_matrix, translation_vector)
    transformed_points.append(transformed_point)
    print("Transformed 3D Point (in color stream coordinates):", transformed_point)
    print(point)

# 출력된 변환된 3D 좌표 리스트
print("All Transformed Points:", transformed_points)